# Wrapping a template library

A template library is a library where there are only template classes that can be instantiated.
Wrapping such libraries therefore requires **AutoWIG** to be able to consider various *C++* template classes instantiations during the `Parse` step.
It is therefore required to install the `pyclanglite` `parser`.

In [8]:
%%bash
conda install -y -q python-clanglite -c statiskit

Fetching package metadata .........
Solving package specifications: ..........

# All requested packages already installed.
# packages in environment at /home/pfernique/.miniconda2/envs/autowig:
#
python-clanglite          3.8.0                    py27_0    statiskit


Using Anaconda Cloud api site https://api.anaconda.org


The **Standard Template Library (STL)** library is a *C++* library that provides a set of common *C++* template classes such as containers and associative arrays.
These classes can be used with any built-in or user-defined type that supports some elementary operations (e.g. copying, assignment).
It is divided in four components called algorithms, containers, functional and iterators.
**STL** containers (e.g. `std::vector`, `std::set`) are used in many *C++* libraries.
In such a case, it does not seem relevant that every wrapped *C++* library contains wrappers for usual **STL** containers (e.g. `std::vector< double >`, `std::set< int >`).
We therefore proposed *Python* bindings for sequence containers (i.e. `pair`, `array`, `vector`, `deque`, `forward_list` and `list` of the `std` namespace) and associative containers (`set`, `multiset`, `map`, `multimap`, `unordered_set`, `unordered_multiset`, `unordered_map` and `unordered_multimap` of the `std` namespace).
These template instantiations are done for *C++* fundamental types (`bool`, `signed char`, `unsigned char`, `char`, `wchar_t`, `int` (with sign modifiers `signed` and `signed` combined or not with size modifiers `short`, `long` and `long long`), `float`, `double`, `long double`) and strings (`string`, `wstring` of the `std` namespace).
For ordered associative containers both `std::less` and `std::greater` comparators are used.
We here only illustrate the procedure on the `std::vector` template class.
For the complete procedure refers to the `AutoWIG.py` file situed at the root of the **PySTL** [repository](https://github.com/StatisKit/PySTL).

The **PySTL** GitHub repository must be cloned into the **PySTL** directory.

In [1]:
%%bash
git clone https://github.com/StatisKit/PySTL.git

Cloning into 'PySTL'...


This repository already has wrappers, we therefore need to remove them.

In [2]:
from path import path
srcdir = path('PySTL')/'src'/'py'
for wrapper in srcdir.walkfiles('*.cpp'):
    wrapper.unlink()
for wrapper in srcdir.walkfiles('*.h'):
    wrapper.unlink()
wrapper = srcdir/'stl'/'vector'/'_vector.py'
if wrapper.exists():
    wrapper.unlink()

Then, to install and compile the *C++* library we use available **Conda** recipes.

In [3]:
%%bash --err error
conda build -q PySTL/conda/libpystl -c statiskit
conda install -y -q libpystl --use-local -c statiskit

scons: Reading SConscript files ...
scons: done reading SConscript files.
scons: Building targets ...
Install file: "build/cpp/array.h" as "/home/pfernique/.miniconda2/envs/_build_placehold_placehold_placehold_placehold_/include/pystl/array.h"
Install file: "build/cpp/deque.h" as "/home/pfernique/.miniconda2/envs/_build_placehold_placehold_placehold_placehold_/include/pystl/deque.h"
Install file: "build/cpp/forward_list.h" as "/home/pfernique/.miniconda2/envs/_build_placehold_placehold_placehold_placehold_/include/pystl/forward_list.h"
Install file: "build/cpp/map.h" as "/home/pfernique/.miniconda2/envs/_build_placehold_placehold_placehold_placehold_/include/pystl/map.h"
Install file: "build/cpp/multimap.h" as "/home/pfernique/.miniconda2/envs/_build_placehold_placehold_placehold_placehold_/include/pystl/multimap.h"
Install file: "build/cpp/multiset.h" as "/home/pfernique/.miniconda2/envs/_build_placehold_placehold_placehold_placehold_/include/pystl/multiset.h"
Install file: "build/cpp

As presented below, in order to wrap a template library, the user needs to write headers containing aliases for desired template class instantiations.

In [4]:
%%bash
pygmentize PySTL/src/cpp/vector.h

#include <vector>
#include <string>

namespace pystl
{
    namespace vector
    {
        typedef std::vector< bool > Bool;
        typedef std::vector< char > Char;
        typedef std::vector< signed char > SignedChar;
        typedef std::vector< unsigned char > UnsignedChar;
        typedef std::vector< int > Int;
        typedef std::vector< short > Short;
        typedef std::vector< short int > ShortInt;
        typedef std::vector< long > Long;
        typedef std::vector< long int > LongInt;
        typedef std::vector< long long > LongLong;
        typedef std::vector< long long int > LongLongInt;
        typedef std::vector< signed > Signed;
        typedef std::vector< signed  int > SignedInt;
        typedef std::vector< signed short > SignedShort;
        typedef std::vector< signed short int > SignedShortInt;
        typedef std::vector< signed long > SignedLong;
        typedef std::vector< signed long int > SignedLongInt;
        typedef std::vector< signed long long >

Once these preliminaries done, we can proceed to the actual generation of wrappers for the **PySTL** library.
For this, we import **AutoWIG** and create an empty Abstract Semantic Graph (ASG).

In [5]:
import autowig
asg = autowig.AbstractSemanticGraph()

We need then to install the *C++* headers. This is done using the `cpp` target in **SCons**.

In [6]:
%%bash
scons cpp -C PySTL

scons: Entering directory `/home/pfernique/Desktop/AutoWIG/doc/examples/PySTL'
scons: Reading SConscript files ...
scons: done reading SConscript files.
scons: Building targets ...
Install file: "build/cpp/array.h" as "/home/pfernique/.miniconda2/envs/autowig/include/pystl/array.h"
Install file: "build/cpp/deque.h" as "/home/pfernique/.miniconda2/envs/autowig/include/pystl/deque.h"
Install file: "build/cpp/forward_list.h" as "/home/pfernique/.miniconda2/envs/autowig/include/pystl/forward_list.h"
Install file: "build/cpp/map.h" as "/home/pfernique/.miniconda2/envs/autowig/include/pystl/map.h"
Install file: "build/cpp/multimap.h" as "/home/pfernique/.miniconda2/envs/autowig/include/pystl/multimap.h"
Install file: "build/cpp/multiset.h" as "/home/pfernique/.miniconda2/envs/autowig/include/pystl/multiset.h"
Install file: "build/cpp/set.h" as "/home/pfernique/.miniconda2/envs/autowig/include/pystl/set.h"
Install file: "build/cpp/vector.h" as "/home/pfernique/.miniconda2/envs/autowig/include


scons: warning: Ignoring missing SConscript 'build/py/vector/SConscript'
File "/home/pfernique/Desktop/AutoWIG/doc/examples/PySTL/SConstruct", line 78, in <module>


Once the headers habe been installed in the system, we parse headers with relevant compilation flags.

In [7]:
import sys
autowig.parser.plugin = 'pyclanglite'
asg = autowig.parser(asg,
                     [sys.prefix + '/include/pystl/vector.h'],
                     flags = ['-x', 'c++', '-std=c++11',
                              '-I' + sys.prefix + '/include'],
                     bootstrap = 1,
                     silent = True)

ImportError: /home/pfernique/.miniconda2/envs/autowig/lib/python2.7/site-packages/clanglite/../../../libclanglite.so: undefined symbol: _ZNK5clang15DeclarationName11getAsStringB5cxx11Ev

Since most of **AutoWIG** guidelines are respected, the `default` `controller` implementation is thus suitable.

In [ ]:
autowig.controller.plugin = 'default'
asg = autowig.controller(asg)

In order to wrap the chosen instations of the `std::vector` template class, we need to select the `boost_python_internal` `generator` implementation.

In [ ]:
autowig.generator.plugin = 'boost_python_internal'
wrappers = autowig.generator(asg,
                  module = './PySTL/src/py/_vector.cpp',
                  prefix = '_vector_wrapper_',
                  decorator = './PySTL/src/py/stl/vector/_vector.py',
                  closure = True)

The wrappers are only generated in-memory.
It is therefore needed to write them on the disk to complete the process.

In [ ]:
wrappers.write()

Here is an example of the generated wrappers.
We here present the wrappers for the `std::vector< int >` class.

In [ ]:
asg['::pystl::vector::Int'].qualified_type.desugared_type.unqualified_type.boost_python_export

Once the wrappers are written on disk, we need to compile and install the *Python* bindings.

In [ ]:
%%bash --err error
conda build -q PySTL/conda/python-stl-vector -c statiskit
conda install -y -q python-stl-vector --use-local -c statiskit

Finally, we can hereafter use the *C++* library in the *Python* interpreter.

In [ ]:
from stl import vector
v = vector.Int()
v.push_back(-1)
v.push_back(0)
v.push_back(1)
v

In [ ]:
list(v)

In [ ]:
v[0]

In [ ]:
v[0] = -2
v[0]

In [ ]:
vector.Int([0, 1])